In [1]:
!pip install BeautifulSoup4
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Extracting data set from wikipedia

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

data = []
columns = []

table = soup.find(class_='wikitable')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
        
    if (index == 0):
        columns = section
    else:
        data.append(section)
        
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove Boroughs that are 'Not assigned'

In [3]:
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Combining neighborhood into one row separated by commas

In [4]:
canada_df = canada_df.groupby(['Postal Code','Borough','Neighborhood']).size().reset_index()
canada_df.drop(columns=[0],inplace =True)

canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Shape

In [5]:
canada_df.shape

(103, 3)

### Get data latitude/longitude data from csv

In [7]:
read_file = pd.read_csv("http://cocl.us/Geospatial_data")

read_file.drop(columns='Postal Code', inplace = True)

canada_df = pd.concat([canada_df, read_file],axis=1, sort =False)
'''
lat_long_coord_df.columns = ["Postcode", "Latitude", "Longitude"]
if(lat_long_coord_df.index.name != 'Postcode'):
    lat_long_coord_df = lat_long_coord_df.set_index('Postcode')
'''
canada_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476
